# Reconocimiento de emociones por imágenes

Este proyecto tiene como objetivo crear un modelo de red neuronal para reconocer si una persona está feliz o no en una imagen. Usaremos el dataset FER-2013 para entrenar un modelo multi-perceptrón y luego haremos predicciones con nuevas imágenes. El modelo será entrenado para clasificar las emociones en dos categorías: "Feliz" y "No Feliz".

## Importar Librerías Necesarias

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import cv2
import os

## Cargar y Preprocesamiento de los Datos

In [ ]:
# Carga del dataset FER-2013
data = pd.read_csv('/content/train/dataset/fer2013.csv')

# Filtrar las imágenes etiquetadas como "feliz"
happy_data = data[data['emotion'] == 3].copy()
not_happy_data = data[data['emotion'] != 3].copy()

# Convertir todas las etiquetas "feliz" a 1 y las demás a 0
happy_data.loc[:, 'emotion'] = 1
not_happy_data.loc[:, 'emotion'] = 0

# Combinar los dos dataframes
binary_data = pd.concat([happy_data, not_happy_data])

# Función para preprocesar las imágenes
def preprocess_images(images):
    images = images.apply(lambda x: np.fromstring(x, sep=' '))
    images = np.stack(images, axis=0)
    images = images.reshape((-1, 48, 48, 1))
    images = images / 255.0  # Normalizar las imágenes
    return images

X = preprocess_images(binary_data['pixels'])
y = binary_data['emotion'].values

# Convertir las etiquetas a categóricas (binarias)
y = to_categorical(y, num_classes=2)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Construccion del Modelo y Entrenamiento

In [ ]:
# Definir el modelo de red neuronal multi-perceptrón
model = Sequential([
    Flatten(input_shape=(48, 48, 1)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Dos neuronas en la capa de salida
])

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Guardar el modelo
model.save('emotion_recognition_model.keras')

Epoch 1/10
898/898 [==============================] - 13s 13ms/step - loss: 0.5440 - accuracy: 0.7464 - val_loss: 0.5243 - val_accuracy: 0.7445
Epoch 2/10
898/898 [==============================] - 13s 14ms/step - loss: 0.5039 - accuracy: 0.7583 - val_loss: 0.5621 - val_accuracy: 0.6967
Epoch 3/10
898/898 [==============================] - 9s 10ms/step - loss: 0.4950 - accuracy: 0.7676 - val_loss: 0.5388 - val_accuracy: 0.7487
Epoch 4/10
898/898 [==============================] - 9s 10ms/step - loss: 0.4778 - accuracy: 0.7795 - val_loss: 0.4775 - val_accuracy: 0.7777
Epoch 5/10
898/898 [==============================] - 6s 7ms/step - loss: 0.4684 - accuracy: 0.7840 - val_loss: 0.4834 - val_accuracy: 0.7738
Epoch 6/10
898/898 [==============================] - 9s 10ms/step - loss: 0.4633 - accuracy: 0.7862 - val_loss: 0.4847 - val_accuracy: 0.7697
Epoch 7/10
898/898 [==============================] - 7s 8ms/step - loss: 0.4634 - accuracy: 0.7889 - val_loss: 0.4641 - val_accuracy: 0.7845

## Prueba de Rendimiento

In [ ]:
# Cargar el modelo guardado
model = tf.keras.models.load_model('emotion_recognition_model.keras')

# Evaluar el rendimiento del modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {accuracy * 100:.2f}%')

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
print('Matriz de confusión:')
print(conf_matrix)

# Calcular el reporte de clasificación
class_report = classification_report(y_true_classes, y_pred_classes, target_names=['No Feliz', 'Feliz'])
print('Reporte de clasificación:')
print(class_report)

225/225 [==============================] - 2s 5ms/step - loss: 0.4662 - accuracy: 0.7834
Precisión en el conjunto de prueba: 78.34%
225/225 [==============================] - 1s 5ms/step
Matriz de confusión:
[[4902  444]
 [1111  721]]
Reporte de clasificación:
              precision    recall  f1-score   support

    No Feliz       0.82      0.92      0.86      5346
       Feliz       0.62      0.39      0.48      1832

    accuracy                           0.78      7178
   macro avg       0.72      0.66      0.67      7178
weighted avg       0.77      0.78      0.77      7178



## Predecir Nuevas Imágenes

In [ ]:
# Cargar el modelo guardado
model = tf.keras.models.load_model('emotion_recognition_model.keras')

# Función para preprocesar nuevas imágenes
def preprocess_new_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (48, 48))
    img = img / 255.0
    img = img.reshape(1, 48, 48, 1)
    return img

# Cargar y preprocesar una nueva imagen
new_image_path = '/content/validation/images/happy_02.jpg'
new_image = preprocess_new_image(new_image_path)

# Hacer la predicción
prediction = model.predict(new_image)
predicted_class = np.argmax(prediction)

# Mostrar el resultado
emotions = ['No Feliz', 'Feliz']
print(f'Emoción predicha: {emotions[predicted_class]}')

1/1 [==============================] - 0s 106ms/step
Emoción predicha: Feliz
